<a href="https://colab.research.google.com/github/spingtseng/Basic-Machine-Learning/blob/main/ML05_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from zipfile import ZipFile
f = ZipFile("/content/drive/MyDrive/news/chinese_news_test.zip 的副本")
f.extractall()
f = ZipFile("/content/drive/MyDrive/news/chinese_news_trans.zip 的副本")
f.extractall()

In [ ]:
import os
import glob
import pandas as pd
def process(base):
    datas = {
        "content":[],
        "ans":[]
    }
    # os.path.join 將多個path組合後返回
    # * 找出所有
    dirs = glob.glob(os.path.join(base, "*"))    
    for d in dirs:
        # 所有的d (環境、交通....)
        fns1 = os.path.join(d, "*.txt")
        fns2 = os.path.join(d, "*.TXT")
        fns = glob.glob(fns1) + glob.glob(fns2)
        for fn in fns:
            with open(fn, "r", encoding="utf-8") as f:
                datas["content"].append(f.read())
            datas["ans"].append(os.path.split(d)[-1])
    df = pd.DataFrame(datas)
    return df
train_df = process("chinese_news_trans")
test_df = process("chinese_news_test")

In [ ]:
train_df

,content,ans
0,日月光華 -- FDU_E.S.E.精華區文章閱讀\n-------------------...,環境
1,日月光華 -- FDU_E.S.E.精華區文章閱讀\n-------------------...,環境
2,日月光華 -- FDU_E.S.E.精華區文章閱讀\n-------------------...,環境
3,日月光華 -- FDU_E.S.E.精華區文章閱讀\n-------------------...,環境
4,日月光華 -- FDU_E.S.E.精華區文章閱讀\n-------------------...,環境
...,...,...
2632,南已有７６人死於艾滋病\n新華社貝爾格萊德５月２３日電南聯邦衛生局２３...,醫藥
2633,英研究用血小板輸送藥物治病\n新華社倫敦５月１４日電（記者王國瑞）英國...,醫藥
2634,新型肝藥馬洛替酯通過國家鑑定\n新華社北京５月１１日電（通訊員蘇文）國...,醫藥
2635,天津一些企業摸索控制醫療費開支新辦法\n新華社天津５月１７日電（記者孫...,醫藥


In [ ]:
u = train_df["ans"].unique()
name2cat = {name:i for i, name in enumerate(u)}
cat2name = {i:name for i, name in enumerate(u)}
y_test = test_df["ans"].replace(name2cat)
y_train

0       0
1       0
2       0
3       0
4       0
       ..
2632    9
2633    9
2634    9
2635    9
2636    9
Name: ans, Length: 2637, dtype: int64

In [ ]:
# 載入大字典(jieba)
from urllib.request import urlretrieve
url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
urlretrieve(url, "dict.text.big")

('dict.text.big', <http.client.HTTPMessage at 0x7fb458001450>)

In [ ]:
import jieba
# 插入大字典
jieba.set_dictionary("dict.text.big")
def newscut(n):
  return " " .join(jieba.cut(n))
x_train = train_df["content"].apply(newscut)
x_test = test_df["content"].apply(newscut)
x_test

Building prefix dict from /content/dict.text.big ...
Dumping model to file cache /tmp/jieba.u39b6bb440e67e760fa0e9a4a04f130c8.cache
Loading model cost 1.678 seconds.
Prefix dict has been built successfully.


0      日月 光華   --   FDU _ E . S . E . 精華區 文章 閱讀 \n --...
1      日月 光華   --   FDU _ E . S . E . 精華區 文章 閱讀 \n --...
2      日月 光華   --   FDU _ E . S . E . 精華區 文章 閱讀 \n --...
3      日月 光華   --   FDU _ E . S . E . 精華區 文章 閱讀 \n --...
4      日月 光華   --   FDU _ E . S . E . 精華區 文章 閱讀 \n --...
                             ...                        
96     日月 光華   --   Health 精華區 文章 閱讀 \n -------------...
97     日月 光華   --   Health 精華區 文章 閱讀 \n -------------...
98     日月 光華   --   Health 精華區 文章 閱讀 \n -------------...
99     日月 光華   --   Health 精華區 文章 閱讀 \n -------------...
100    日月 光華   --   Health 精華區 文章 閱讀 \n -------------...
Name: content, Length: 101, dtype: object

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_count = vec.fit_transform(x_train)
x_test_count = vec.transform(x_test)

In [ ]:
vec.vocabulary_
x_train_count
# print(x_test_count)

<2637x76399 sparse matrix of type '<class 'numpy.int64'>'
	with 448436 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha = 0.1)
clf.fit(x_train_count, y_train)

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [ ]:
from sklearn.metrics import accuracy_score, log_loss
pre = clf.predict(x_test_count)
print(accuracy_score(pre, y_test))
prob = clf.predict_proba(x_test_count)
print(log_loss(y_test, prob))

1.0
9.192866490039826e-15


In [ ]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pre)
pd.DataFrame(mat,
       index = [name + "(原本)" for name in u],
       columns = [name + "(預測)" for name in u])

,環境(預測),政治(預測),交通(預測),計算機(預測),教育(預測),軍事(預測),經濟(預測),體育(預測),藝術(預測),醫藥(預測)
環境(原本),10,0,0,0,0,0,0,0,0,0
政治(原本),0,11,0,0,0,0,0,0,0,0
交通(原本),0,0,10,0,0,0,0,0,0,0
計算機(原本),0,0,0,10,0,0,0,0,0,0
教育(原本),0,0,0,0,10,0,0,0,0,0
軍事(原本),0,0,0,0,0,10,0,0,0,0
經濟(原本),0,0,0,0,0,0,10,0,0,0
體育(原本),0,0,0,0,0,0,0,10,0,0
藝術(原本),0,0,0,0,0,0,0,0,10,0
醫藥(原本),0,0,0,0,0,0,0,0,0,10


In [ ]:
n = input("write:")
test = vec.transform([newscut(n)])
prob = clf.predict_proba(test)[0]
for i in range(len(u)):
  print(u[i], "的機率：", round(prob[i], 3))

write:二刀流可不是大谷翔平的專利，光芒隊大物新秀麥凱(Brendan McKay)去年（2020）因傷被迫關機，本季春訓再次回歸，他透露自己將持續以二刀流身份出賽。  麥凱在2年前（2019）登上大聯盟，當時在13次出賽中主投49局，繳出防禦5.14的成績，打擊方面則是10支2，包含一發全壘打，隔年光芒隊要他更加著重於投球的訓練，但在他7月時被新冠病毒檢驗為陽性後一切又變了調，他錯過了小聯盟夏季聯賽的剩餘賽事，康復後在牛棚試投幾球還是覺得肩膀不適。  去年8月麥凱接受核磁共振檢驗，發現他由於肩關節唇撕裂導致肩後發炎，隨後進行了手術而提前結束賽季，今年春訓比起投球，他提升了更多打擊上的訓練比重，過去在小聯盟559個打席中繳出.214打擊率和.697的攻擊指數，附帶15發全壘打看似平庸，但在大學時期他可是打擊三圍.328/.430/.536的強打者，他表示自己在12月中旬開始練習揮棒，先前的工作只是讓自己回復比賽時的狀態，並透露現在的揮棒感覺相當不賴。  麥凱預計將在3月底的熱身賽以投手身份登板，對新秀排名第72、同時也是十大左投第10名的他來說是個證明自己的好機會，然而日前又因為將狗狗玩具上的標籤剪下時意外劃傷自己的左手食指，使得他不得不將投球訓練暫停，麥凱隨後向媒體表示自己將儘快復原重返軌道。  麥凱也透露自己花了很多時間看熱身賽比賽，並時常留意同為投打二刀流的大谷翔平，他得知大谷既可以飆出三位數火球，又能在打擊區敲全壘打，在被記者問到如何看待自己現在是個投手還是打者時，他回答：「現在依舊兩者皆是。」
環境 的機率： 0.0
政治 的機率： 0.0
交通 的機率： 0.0
計算機 的機率： 0.0
教育 的機率： 0.0
軍事 的機率： 0.0
經濟 的機率： 0.0
體育 的機率： 1.0
藝術 的機率： 0.0
醫藥 的機率： 0.0


In [ ]:
# 安裝opencc程式碼 中文繁簡轉換
# https://pypi.org/project/opencc-python-reimplemented/
!pip install opencc-python-reimplemented

In [ ]:
from opencc import OpenCC
cc = OpenCC('s2twp')  # convert from Simplified Chinese to Traditional Chinese
# can also set conversion by calling set_conversion
# cc.set_conversion('s2tw')
to_convert = '开放中文转换'
converted = cc.convert(to_convert)
converted

'開放中文轉換'